In [1]:
# %load hw1.py
import urllib.request
from numpy.lib.function_base import append  # lib that handles URLs
import pandas as pd
import re
import jieba
import jieba.analyse
import math

In [2]:
target_url = "https://raw.githubusercontent.com/cjwu/cjwu.github.io/master/courses/nlp/hw1-dataset.txt"
data = urllib.request.urlopen(target_url)

with open('output.txt', 'a+') as f:
    for line in data:
        decoded_line = line.decode("utf-8")
        f.write(decoded_line)

In [3]:
df = pd.read_fwf('./output.txt',header = None)
df.columns = ['content']
df.to_csv('./log.csv', encoding = 'utf-8',index = True)

In [4]:
class WordSegment:
    def __init__(self,texts,stop_words_path,dicts_path=""):
        self.jieba = jieba
        self.texts = texts
        self.dicts_path = dicts_path
        self.stop_words_path = stop_words_path
        
    def read_dictionary(self): 
        #讀取類別
        if self.dicts_path:
            for dic in self.dicts_path:
                self.jieba.load_userdict(dic)

    def read_stop_words(self):
        #讀取stopword
        self.stop_words = set()
        if stop_words_path:
            with open(stop_words_path,encoding = 'UTF-8') as file:
                self.stop_words = self.stop_words | set(map(str.strip,file.readlines()))
        
    def remove_stop_words(self):
        # 去除繁體中文以外的英文、數字、符號
        #jieba.set_dictionary('./dict.txt.big')
        rule = re.compile(r"[^\u4e00-\u9fa5]")
        self.texts = [list(self.jieba.cut(rule.sub('', text))) for text in self.texts]
        for idx, speech in enumerate(self.texts):
            self.texts[idx] = ' '.join([word for word in speech if word.strip() not in self.stop_words])

In [8]:
topic = {}
with open('log.csv', newline='') as f:
    contents = ""
    for i,line in enumerate(f.readlines()):
        if i==0:continue
        ind, content = line.split(",",1)
        contents = content
        topic[ind] = content
#print(topic)

In [7]:
jieba.setLogLevel(jieba.logging.INFO)
stop_words_path = './stop_words.txt'

ws = WordSegment(topic.values(),stop_words_path)
ws.read_stop_words()
ws.remove_stop_words()
text_files = [text.split(" ") for text in ws.texts]